In [1]:
import pydot
import networkx as nx
import matplotlib.pyplot as plt
from collections import defaultdict
from networkx.drawing.nx_pydot import graphviz_layout

In [2]:
class Graph:
    def __init__(self, vertices, edges, weighted = False):
        self.vertices = vertices
        self.weighted = weighted
        self.visualization = edges
        self.adjList = defaultdict(list)
        self.costDict = {}
        self.create_adj_list(edges)
        self.create_edge_heuristics(edges)
    
    def visualize_graph(self):
        nx_graph = nx.Graph()
        if (self.weighted == True):
            nx_graph.add_weighted_edges_from(self.visualization)
            costs = nx.get_edge_attributes(nx_graph, 'weight')
            pos = graphviz_layout(nx_graph, prog = "dot", root = self.vertices[0])
            nx.draw_networkx_edge_labels(nx_graph, pos, edge_labels = costs, font_color = 'red')
            nx.draw_networkx(nx_graph, pos, with_labels = "True", alpha = 0.9, node_color = 'cyan')
        else:
            pos = graphviz_layout(nx_graph, prog = "dot", root = self.vertices[0])
            nx_graph.add_edges_from(self.visualization)
            nx.draw_networkx(nx_graph, pos, with_labels = "True")
        plt.show()
        
    def create_adj_list(self, edges):
        for edge in edges:
            source = edge[0]
            destination = edge[1]
            self.adjList[source].append(destination)
            self.adjList[destination].append(source)
            
    def print_adj_list(self):
        for item in self.adjList.items():
            print(item)
    
    def create_edge_heuristics(self, edges):
        if (self.weighted == True):
            for edge in edges:
                dict_key = (edge[0], edge[1])
                dict_value = edge[2]
                self.costDict[dict_key] = dict_value